In [ ]:
%matplotlib inline


# Using the ExecuTorch Developer Tools to Profile a Model

Please update your link to <https://pytorch.org/executorch/main/tutorials/devtools-integration-tutorial.html>. This URL will be deleted after v0.4.0.
